
<h2><center><font color='black'>   Taking a Deep-Learning Dive with Keras </font></center></h2>

<img src='imgs/Deep_Dive_Keras_2.jpg' align='middle'/>

<img src='imgs/cnn1.png'/>

<img src='imgs/cnn_family_a.png'/>

<img src='imgs/cnn_family.png'/>
<img src='imgs/many_architectures.png'/>
* bubble size is # of parameters
* AlexNet  (62 million parameters) (final layer 6x 6 x 256 x 2096) ~ > 1/2 are coming from FC layer

<img src='imgs/input.png'/>

<img src='imgs/conv1.png'/>

<img src='imgs/conv_title.png'/>

<img src='imgs/3D_Convolution_Animation.gif'/>

**Definition of Convolution:**
Integral that expresses overlap of one function as it is shifted over another

**In terms of CNNs:**
The input is one function, and the kernel is the other. We can implement the integral as 
a summation over a finite # of array elements.  The output is called the feature map (which we can think of as a weighted sum)

<img src='imgs/conv_a.png'/>
<img src='imgs/conv_bb.png'/>

<img src='imgs/conv_gif_a.png'/>
<img src='imgs/conv_filt_b.png'/>


<img src='imgs/conv_layers.png'/>


<img src='imgs/conv_filter.png'/>

<img src='imgs/max_pool.png'/>


### Max-Pooling benefits: 

- Reduces parameters by 75% 
- Helps us to generalize our model: Provides an 'abstracted' form of our features

<img src='imgs/relu_1.png'/>

<img src='imgs/relu2.png'/>


<img src='imgs/fc.png'/>

<img src='imgs/all_layers.png'/>

Credits: [Yosinski](http://yosinski.com)




In [2]:
from keras import backend as K
import keras.utils.np_utils

from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, ZeroPadding1D
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
import numpy as np
import pandas as pd

from IPython.display import SVG
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import sys

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

##  And now,  CNN in Keras! 


In [3]:
import keras.backend as K
K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

#### Macroarchitecture of VGG16
<img src='imgs/VGG.png'/>

In [4]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512,(3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [5]:
!curl https://raw.githubusercontent.com/torch/tutorials/master/7_imagenet_classification/synset_words.txt -o synset_words.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31675  100 31675    0     0   104k      0 --:--:-- --:--:-- --:--:--  104k


[Download weights](https://github.com/fchollet/deep-learning-models/releases/tag/v0.1)

In [6]:
model = VGG_16('~/Downloads/vgg16_weights_th_dim_ordering_th_kernels.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [7]:
synset = pd.read_csv('synset_words.txt', skipinitialspace=True, names = ['synset', 'words'])

In [8]:
def prepare_image(image_path):
    im = cv2.resize(cv2.imread(image_path), (224, 224)).astype(np.float32)
    # subtracting means across the color channels
    # (The mean pixel values are taken from the VGG authors)
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    return im

<img src='imgs/dog2.jpg'/>

In [9]:
img = prepare_image('imgs/dog2.jpg')
out = model.predict(img)
y_pred = np.argmax(out)

print (y_pred)
print (synset.loc[y_pred].synset)

259
n02112018 Pomeranian


Close enough! :) 

<img src='imgs/sloth.jpg'/>

In [10]:
img = prepare_image('imgs/sloth.jpg')
out = model.predict(img)
y_pred = np.argmax(out)

print (y_pred)
print (synset.loc[y_pred].synset)

968
n07930864 cup


<img src='imgs/dress_two.png'/>

### Working with CNNs &  [CIFAR dataset ](https://www.cs.toronto.edu/~kriz/cifar.html)
- The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes

<img src ='imgs/cifar.png'/>

In [11]:
from keras.datasets import cifar10
 
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [9]:
X_train.shape

(50000, 3, 32, 32)

In [65]:
from keras.utils.np_utils import to_categorical
y_test=to_categorical(y_test)
y_train=to_categorical(y_train)

###  Let's work with VGG16 model again

Lets import the model this time



In [31]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights=None, include_top=False)
model_vgg16_conv.summary()

#Create your own input format 
input = Input(shape=(3,32,32),name = 'image_input')  # our input shape: (3,32,32)

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)  # Using the Functional API

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, None, None)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, None, None)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, None, None)   0         
__________

/Users/julialintern/anaconda/envs/ipython_env/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=/image_inp...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 3, 32, 32)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              2101248   
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 10)                40970     
Total params: 33,638,218
Trainable params: 33,638,218
Non-trainable params: 0
________________________________________________________________

In [32]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
my_model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])



In [ ]:
cb_a=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
# lets look at our results while we are at it
cb_b=keras.callbacks.RemoteMonitor(root='http://localhost:9000')


my_model.fit(X_train, y_train,
          batch_size=128, epochs=10, verbose=1,
          validation_data=(X_test, y_test),callbacks=[cb_a,cb_b])

### Code-Along! 

Let's repeat the MNIST training by leveraging [Transfer Learning](http://cs231n.github.io/transfer-learning/)

1) Reinstantiate your VGG16 model: this time set weights='imagenet'
2) Freeze all previous layers